In [1]:
from inference_sdk import InferenceHTTPClient
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt
import csv
import os
import numpy as np
import math
from shapely.geometry import Point, Polygon

# Initialize the client
CLIENT = InferenceHTTPClient(
    api_url="",
    api_key=""
)

# Define input and output directories
input_dir = "1CELL"
output_dir = "FOUND"
os.makedirs(output_dir, exist_ok=True)  # Create the output directory if it doesn't exist

# Loop over each image file in the input directory
for filename in os.listdir(input_dir):
    if filename.endswith(".bmp"):  # Process only BMP files
        image_path = os.path.join(input_dir, filename)
        output_image_path = os.path.join(output_dir, f"post-processed_{filename}")
        csv_file_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}.csv")

        # Perform inference on the image
        result = CLIENT.infer(image_path, model_id="cr-vor6k/2")

        # Check if the result contains predictions
        if 'predictions' not in result or not result['predictions']:
            print(f"No predictions found in {filename}")
            continue

        # Open the image
        image = Image.open(image_path)
        draw = ImageDraw.Draw(image)

        # Get image dimensions
        width, height = image.size
        print(f"Processing {filename}: Width = {width}, Height = {height}")

        # Larger font size for cell number
        try:
            font = ImageFont.truetype("arial.ttf", 32)
        except IOError:
            font = ImageFont.load_default()

        # Prepare data for CSV with only Cell Number, Pink Line Length, Yellow Line Length, and Area
        csv_data = []

        # Iterate over each prediction and draw the polygon using every point
        for i, prediction in enumerate(result['predictions']):
            points_dict = prediction.get('points', [])
            cell_number = i + 1

            # Convert list of dictionaries to list of (x, y) tuples for every point in the polygon
            points = [(point['x'], point['y']) for point in points_dict]
            int_points = [(int(x), int(y)) for x, y in points]

            # Draw the polygon outline in red
            if len(int_points) > 2:
                draw.polygon(int_points, outline="red", width=3)

            # Calculate the area of the cell
            polygon = Polygon(int_points)
            area = polygon.area
            print(f"Area of Cell {cell_number} in {filename}: {area:.2f}")

            # Calculate the centroid for placing the number
            centroid_x = sum([p[0] for p in int_points]) // len(int_points)
            centroid_y = sum([p[1] for p in int_points]) // len(int_points)

            # Draw the cell number in the center (centroid) of the cell with a larger font
            draw.text((centroid_x, centroid_y), str(cell_number), fill="white", font=font)

            # Calculate the maximum radius (distance from centroid to furthest point)
            max_radius = float('-inf')
            max_radius_point = None

            for (x, y) in int_points:
                radius = np.sqrt((centroid_x - x) ** 2 + (centroid_y - y) ** 2)
                if radius > max_radius:
                    max_radius = radius
                    max_radius_point = (x, y)

            # Extend the pink line across the cell until it reaches the polygon boundary on both sides
            if max_radius_point:
                # Calculate the vector from the centroid to the max_radius_point
                vector_x = max_radius_point[0] - centroid_x
                vector_y = max_radius_point[1] - centroid_y

                # Normalize the vector to move in small steps
                step_size = 1
                vector_length = np.sqrt(vector_x**2 + vector_y**2)
                unit_vector_x = (vector_x / vector_length) * step_size
                unit_vector_y = (vector_y / vector_length) * step_size

                # Define a maximum iteration count to prevent infinite loops
                max_iterations = 1000

                # Draw the line from the centroid in the positive direction
                current_x, current_y = centroid_x, centroid_y
                end_x_positive, end_y_positive = None, None
                for _ in range(max_iterations):
                    current_x += unit_vector_x
                    current_y += unit_vector_y
                    if not polygon.contains(Point(current_x, current_y)):
                        end_x_positive, end_y_positive = int(current_x), int(current_y)
                        break

                # Draw the line from the centroid in the negative direction
                current_x, current_y = centroid_x, centroid_y
                end_x_negative, end_y_negative = None, None
                for _ in range(max_iterations):
                    current_x -= unit_vector_x
                    current_y -= unit_vector_y
                    if not polygon.contains(Point(current_x, current_y)):
                        end_x_negative, end_y_negative = int(current_x), int(current_y)
                        break

                # Draw the pink line and calculate its length
                if end_x_positive and end_y_positive and end_x_negative and end_y_negative:
                    draw.line([(end_x_negative, end_y_negative), (end_x_positive, end_y_positive)], fill="pink", width=2)
                    pink_length = np.sqrt((end_x_positive - end_x_negative)**2 + (end_y_positive - end_y_negative)**2)
                    print(f"Pink line length for Cell {cell_number} in {filename}: {pink_length:.2f}")

                    # Midpoint of the pink line
                    midpoint_x = (end_x_negative + end_x_positive) // 2
                    midpoint_y = (end_y_negative + end_y_positive) // 2
                    draw.ellipse([(midpoint_x - 3, midpoint_y - 3), (midpoint_x + 3, midpoint_y + 3)], fill="blue", outline="blue")

                    # Define a function to extend line within polygon to prevent infinite loops
                    def extend_line_within_polygon(start_x, start_y, vector_x, vector_y, max_length, polygon, step_size=1):
                        current_x, current_y = start_x, start_y
                        distance = 0
                        for _ in range(max_iterations):
                            current_x += vector_x
                            current_y += vector_y
                            if not polygon.contains(Point(current_x, current_y)):
                                return int(current_x), int(current_y)
                            distance += step_size
                            if distance >= max_length:
                                break
                        return int(current_x), int(current_y)

                    # Set maximum length for the yellow line in each direction
                    yellow_length = 75

                    # Calculate perpendicular vector for yellow line
                    perpendicular_vector_x = -(end_y_positive - end_y_negative)
                    perpendicular_vector_y = end_x_positive - end_x_negative

                    # Normalize the perpendicular vector
                    vector_length = np.sqrt(perpendicular_vector_x**2 + perpendicular_vector_y**2)
                    unit_vector_x = (perpendicular_vector_x / vector_length) * step_size
                    unit_vector_y = (perpendicular_vector_y / vector_length) * step_size

                    # Extend yellow line in both perpendicular directions from the midpoint
                    yellow_end_positive = extend_line_within_polygon(midpoint_x, midpoint_y, unit_vector_x, unit_vector_y, yellow_length, polygon)
                    yellow_end_negative = extend_line_within_polygon(midpoint_x, midpoint_y, -unit_vector_x, -unit_vector_y, yellow_length, polygon)

                    # Draw the yellow line and calculate its length
                    if yellow_end_negative and yellow_end_positive:
                        draw.line([yellow_end_negative, yellow_end_positive], fill="yellow", width=2)
                        yellow_line_length = np.sqrt((yellow_end_positive[0] - yellow_end_negative[0])**2 + (yellow_end_positive[1] - yellow_end_negative[1])**2)
                        print(f"Yellow line length for Cell {cell_number} in {filename}: {yellow_line_length:.2f}")

                # Append only the required data to CSV
                csv_data.append([cell_number, pink_length, yellow_line_length, area])

        # Save CSV file with only the specified columns
        with open(csv_file_path, mode='w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['Cell Number', 'Pink Line Length', 'Yellow Line Length', 'Area'])  # Write header
            csv_writer.writerows(csv_data)  # Write data rows

        print(f"Data saved to {csv_file_path}")

        # Save the annotated image
        image.save(output_image_path)
        print(f"Processed image saved as {output_image_path}")

print("Processing complete for all images.")


Processing Image-17.bmp: Width = 640, Height = 480
Area of Cell 1 in Image-17.bmp: 12682.50
Pink line length for Cell 1 in Image-17.bmp: 161.68
Yellow line length for Cell 1 in Image-17.bmp: 98.08
Area of Cell 2 in Image-17.bmp: 9468.00
Pink line length for Cell 2 in Image-17.bmp: 116.85
Yellow line length for Cell 2 in Image-17.bmp: 102.96
Area of Cell 3 in Image-17.bmp: 3707.00
Pink line length for Cell 3 in Image-17.bmp: 89.40
Yellow line length for Cell 3 in Image-17.bmp: 38.60
Data saved to FOUND/Image-17.csv
Processed image saved as FOUND/post-processed_Image-17.bmp
Processing Image-16.bmp: Width = 640, Height = 480
Area of Cell 1 in Image-16.bmp: 15129.00
Pink line length for Cell 1 in Image-16.bmp: 187.24
Yellow line length for Cell 1 in Image-16.bmp: 104.05
Area of Cell 2 in Image-16.bmp: 10185.50
Pink line length for Cell 2 in Image-16.bmp: 171.21
Yellow line length for Cell 2 in Image-16.bmp: 77.47
Area of Cell 3 in Image-16.bmp: 888.00
Pink line length for Cell 3 in Image-1